
# 📊 Semana 04 — Visualización avanzada e interactiva con Altair + hvPlot
## Dataset real: Online Retail II (UCI) — EDA, interactividad y mini-dashboard

**Objetivo:** trabajar con un dataset real de ventas (≈500k filas) para practicar **Altair** (declarativo) y **hvPlot + Panel** (interactivo rápido).

> 🔒 Versión alumno: enunciados, pistas y tests.


## 0) Preparación del entorno

In [3]:

#%pip -q install altair==5.* vega_datasets==0.9.* hvplot==0.9.* panel==1.* openpyxl==3.*
import warnings; warnings.filterwarnings("ignore")
import pandas as pd, numpy as np, altair as alt, hvplot.pandas, panel as pn, math, sys, time
alt.data_transformers.disable_max_rows()
pn.extension('tabulator')
print("✔ Entorno listo:", pd.__version__, alt.__version__)


ModuleNotFoundError: No module named 'hvplot'

## 1) Carga del dataset (Online Retail II)

In [ ]:

import io, requests
def load_online_retail_ii():
    urls = [
        "https://archive.ics.uci.edu/ml/machine-learning-databases/00502/online_retail_II.xlsx",
        "https://archive.ics.uci.edu/ml/machine-learning-databases/00502/online_retail_II.xlsx?raw=true",
    ]
    for url in urls:
        try:
            r = requests.get(url, timeout=30)
            if r.ok and r.content:
                xls = pd.ExcelFile(io.BytesIO(r.content))
                frames = []
                for name in xls.sheet_names:
                    try:
                        frames.append(xls.parse(name))
                    except Exception:
                        pass
                if frames:
                    return pd.concat(frames, ignore_index=True)
        except Exception as e:
            print("Aviso:", e)
    rng = np.random.default_rng(42)
    n = 50_000
    df = pd.DataFrame({
        "InvoiceNo": rng.integers(100000, 999999, n).astype(str),
        "StockCode": rng.choice(["84029G","85048","20725","22633","22492"], n),
        "Description": rng.choice(["MUG","HEART BOX","JUMBO BAG","METAL SIGN","SET TEA"], n),
        "Quantity": rng.integers(1, 10, n),
        "InvoiceDate": pd.to_datetime("2010-01-01") + pd.to_timedelta(rng.integers(0, 365, n), unit="D"),
        "UnitPrice": rng.uniform(0.5, 30.0, n).round(2),
        "CustomerID": rng.integers(12345, 67890, n).astype("Int64"),
        "Country": rng.choice(["United Kingdom","Germany","France","Spain","Netherlands"], n)
    })
    return df
df_raw = load_online_retail_ii()
df_raw.head()
print("Filas/Columnas:", df_raw.shape)


Aviso: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.


In [ ]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   Invoice      1067371 non-null  object        
 1   StockCode    1067371 non-null  object        
 2   Description  1062989 non-null  object        
 3   Quantity     1067371 non-null  int64         
 4   InvoiceDate  1067371 non-null  datetime64[ns]
 5   Price        1067371 non-null  float64       
 6   Customer ID  824364 non-null   float64       
 7   Country      1067371 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 65.1+ MB


## 2) Limpieza mínima y enriquecimiento


**Tareas:**
1. Elimina filas con `Quantity <= 0` o `UnitPrice <= 0`.
2. Crea `Total` = `Quantity * UnitPrice`.
3. Convierte `InvoiceDate` a tipo fecha y crea columnas `Year`, `Month`, `Day`.
4. Elimina nulos de `CustomerID` (o imputa).

> 💡 *Pistas:* usa `query`, `assign`, `to_datetime`, `dt.year`, `dropna`.


In [ ]:
df = df_raw.copy()
df= df[(df["Quantity"] > 0) & (df["Price"] > 0)]

df["TotalPrice"] = df["Quantity"] * df["Price"]
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"], errors="coerce")


In [ ]:

# ✏️ Tu código aquí
df = df_raw.copy()

df= df[(df["Quantity"] > 0) & (df["Price"] > 0)]

df["Total"] = df["Quantity"] * df["Price"]
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"], errors="coerce")
df["Year"] = df["InvoiceDate"].dt.year
df["Month"] = df["InvoiceDate"].dt.month
df["Day"] = df["InvoiceDate"].dt.day
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Total,Year,Month,Day
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.4,2009,12,1
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0,2009,12,1
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0,2009,12,1
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,100.8,2009,12,1
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.0,2009,12,1


In [ ]:

# ✅ Tests rápidos
assert "Total" in df.columns, "Falta columna Total"
assert df["Quantity"].min() >= 0, "Quantity debe ser >= 0"
assert df["Price"].min() >= 0, "UnitPrice debe ser >= 0"
assert {"Year","Month","Day"}.issubset(df.columns), "Faltan columnas de fecha"
print("✔ Limpieza básica OK")


✔ Limpieza básica OK


## 3) Altair — básicos (EDA declarativo)


**Tareas:**
- Histograma de `Total` (bins=50), con tooltip.
- Barras Top-10 `Description` por suma de `Total`.

> 💡 Usa `mark_bar()`, `transform_bin`, `encode`, `transform_aggregate` y `sort='-y'`.


In [ ]:

# ✏️ Tu código aquí (Altair)

# hist_total = ...
# top_desc = ...

# hist_total | top_desc


## 4) Altair — interactividad (selección y zoom)


**Tareas:**
- Crea un **selector por país** que filtre un scatter `UnitPrice` vs `Quantity` (size = `Total`).
- Añade `interactive()` para permitir zoom/drag.

> 💡 Usa `selection_point(fields=['Country'])` y `transform_filter`.


In [ ]:

# ✏️ Tu código aquí (Altair interactivo)

# country_sel = ...
# scatter = ...

# scatter


## 5) hvPlot — series y rankings rápidos


**Tareas:**
- Calcula ventas diarias y plótalas en línea con `hvplot.line` (+ media móvil 7 días).
- Muestra Top-10 países por facturación con `hvplot.barh`.

> 💡 Usa `groupby` por fecha y `rolling(window=7).mean()`.


In [ ]:

# ✏️ Tu código aquí (hvPlot)

# df_daily = ...
# line = ...
# top_countries = ...

# line + top_countries


## 6) Mini-dashboard con Panel (filtros + vistas)


**Tareas:**
- Crea un `pn.widgets.Select` para `Country` (incluye opción 'ALL').
- Muestra: (1) tabla Top-15 productos (`Total`), (2) línea de ventas por mes, (3) barras por mes.
- Vincula el filtro para actualizar las tres vistas.

> 💡 Usa `@pn.depends(selector)` para reaccionar a cambios.


In [ ]:

# ✏️ Tu código aquí (Panel)

# selector = pn.widgets.Select(...)
# @pn.depends(selector)
# def top_products(country): ...
# @pn.depends(selector)
# def monthly_line(country): ...
# @pn.depends(selector)
# def monthly_bar(country): ...

# dashboard = pn.Column(selector, pn.Row(top_products, monthly_line, monthly_bar))
# dashboard


## 7) Comparativa Altair vs hvPlot


**Altair**: sintaxis declarativa, muy buena para *storytelling* con selecciones y transformaciones.  
**hvPlot**: muy rápido para *prototipar interactividad* directamente desde DataFrames; perfecto combinado con **Panel**.

**Preguntas para reflexionar:**
- ¿Qué te ha resultado más expresivo/rápido para EDA?
- ¿Qué herramienta usarías para un dashboard simple? ¿y para un informe narrativo?


## 8) Tests automáticos de la práctica

In [ ]:

# ✅ Tests (básicos)
assert "Total" in df.columns and df["Total"].ge(0).all(), "Fallo en Total"
assert df["InvoiceDate"].dtype.kind in ("M","m"), "InvoiceDate debe ser datetime"
assert df.groupby("Country")["Total"].sum().shape[0] >= 1, "Groupby por país vacío"
print("✔ Todos los tests básicos OK")



## 9) Cierre
- Has practicado **Altair** y **hvPlot + Panel** con un dataset real.
- Te llevas un **mini-dashboard** y varias vistas interactivas.
- Próximo paso: **storytelling completo** (título → hipótesis → evidencia → insight → acción).
